In [29]:
import pandas as pd
import numpy as np
import csv
import xlrd
import openpyxl
from pathlib import Path
import datetime
import os

### Read in tables from csv

In [32]:
os.getcwd().parent_dir()

AttributeError: 'str' object has no attribute 'parent_dir'

In [28]:

xl_name = r'.Budget Data Tables 2022 v1.0.xlsx'
zeta_raw = pd.read_excel(xl_name, sheet_name= 'Zeta Data')
acc_bal = pd.read_excel(xl_name, sheet_name= 'Account Balance')
bud = pd.read_excel(xl_name, sheet_name= 'Budget')

FileNotFoundError: [Errno 2] No such file or directory: '.Budget Data Tables 2022 v1.0.xlsx'

In [3]:
zeta_raw.tail()

,date,vendor,amount,ownership,category,account
598,2021-07-15,Sp The Klassy Shop,-78.62,shared,#Clothes,Tangerine Credit
599,2021-07-15,Thank You,400.00,shared,#CreditCardPayment,Tangerine Credit
600,2021-07-15,Eft Deposit From Eq Bank,2000.00,shared,#Savings,Shared Tang Check
601,2021-07-15,Eft Deposit From Eq Bank,600.00,shared,#Savings,Shared Tang Check
602,2021-07-15,Bell Canada Ob Montreal Qc,-95.99,shared,#RentInternetUtilities,Tangerine Credit


In [4]:
acc_bal.head()

,Accounts,2019,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,TD - Shared cq,4796.85,5504.23,5018.38,4306.79,3641.07,4786.64,3366.54,NaN,NaN,NaN,NaN,NaN,NaN
1,Tang - Shared cq,10491.20,8314.76,6169.55,6740.49,6723.89,6451.65,3888.13,NaN,NaN,NaN,NaN,NaN,NaN
2,Tang - Gods cq,394.63,334.63,240.00,0.00,778.29,625.15,950.00,NaN,NaN,NaN,NaN,NaN,NaN
3,Tang - sv,41.33,64.70,50.00,62.65,86.82,111.11,50.00,NaN,NaN,NaN,NaN,NaN,NaN
4,Tang - TFSA Inv,5476.75,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
bud.head()

,Categories_Raw,Categories,Group,Flow,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,#VivCheque,Viv's Cheque,Income,In,2486.25,2350,2300,2300,2300,600,4400,NaN,NaN,NaN,NaN,NaN
1,#TimmyCheque,Timmy's Cheque,Income,In,3258,3700,3700,3700,3750,3750,4000,NaN,NaN,NaN,NaN,NaN
2,#BbPay,BB Income,Income,In,0,0,0,100,100,0,0,NaN,NaN,NaN,NaN,NaN
3,#InterestIncome,Interest Income,Income,In,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,#OtherIncome,Other Income,Income,In,10,10,10,10,10,10,10,NaN,NaN,NaN,NaN,NaN


### Turn transaction data into usable format

#### turn date into month

In [6]:

zeta_raw['monthnum'] = zeta_raw['date'].apply(lambda x: x.month) #gets month number from date column
months = {1: "Jan", 2: "Feb", 3: "Mar",4: "Apr",5: "May", 6: "Jun", 7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11:"Nov", 12:"Dec"}
zeta_raw['month'] = zeta_raw['monthnum'].apply(lambda y: months[y]) #gets month name from month number

In [7]:
#group into months and categories
zeta_grp = zeta_raw.groupby(by=['month','category']) 
zeta_sum = zeta_grp.sum().drop(columns = 'monthnum')

In [8]:
zeta_sum.query("category == '#Savings'")

,,amount
month,category,
Apr,#Savings,-100.00
Feb,#Savings,-4400.00
Jan,#Savings,17.11
Jul,#Savings,1900.00
Jun,#Savings,1300.00
Mar,#Savings,-4317.55
May,#Savings,500.00


## Budget table

In [9]:
#unpivot budget table so its a nice data format
bud_melt = pd.melt(bud,['Categories','Categories_Raw','Group','Flow'],['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
bud_melt.dropna(inplace= True)
bud_melt.rename(columns={'value':'budget','variable':'month'},inplace = True)
bud_melt.set_index(['month','Categories_Raw'])

Categories     Group      Flow   budget
month Categories_Raw                                                     
Jan   #VivCheque                Viv's Cheque    Income        In  2486.25
      #TimmyCheque            Timmy's Cheque    Income        In     3258
      #BbPay                       BB Income    Income        In        0
      #InterestIncome        Interest Income    Income        In        0
      #OtherIncome              Other Income    Income        In       10
...                                      ...       ...       ...      ...
Jul   #Twork              Timmy Work Expense      Work       Out        0
      #CreditCardPayment     Pay Credit Card  Transfer  Transfer        0
      #Transfer                     Transfer  Transfer  Transfer        0
      #TaxesPrepaid            Taxes Prepaid    Living       Out      600
      #Savings                        Saving    Saving       Out        0

[217 rows x 4 columns]

# Combine budget and zeta data table

In [10]:
#merge the 2 dfs together
trans = pd.merge(zeta_sum,bud_melt,how = 'outer', left_on = ['month','category'], right_on = ['month','Categories_Raw'])
#rearrange columns
trans.drop('Categories_Raw', inplace = True, axis = 1)
trans.rename(columns = {'amount':'actual','Categories':'category','Group':'group','Flow':'flow'},inplace = True)
trans = trans[['month','category','group','flow','actual','budget']]
trans.fillna(0,inplace = True)

In [11]:
trans

,month,category,group,flow,actual,budget
0,Apr,Alcohol,Leisure,Out,-86.08,50.0
1,Apr,BB Income,Income,In,20.00,100.0
2,Apr,Car,Leisure,Out,-203.94,450.0
3,Apr,Cellphone,Housing,Out,-168.68,50.0
4,Apr,Clothes,Living,Out,-204.54,200.0
...,...,...,...,...,...,...
212,Jul,Public Transportation,Leisure,Out,0.00,20.0
213,Jul,Special Splurge,Leisure,Out,0.00,0.0
214,Jul,Income Taxes,Living,Out,0.00,0.0
215,Jul,Viv's Choice,Leisure,Out,0.00,70.0


In [12]:
#create nonth number column for indexing later
trans['monthnum'] = trans['month'].apply(lambda x: list(months.values()).index(x))+1

In [13]:
#create the function to get the difference between budget and actual
def correct_bud(x):
    bud = x['budget']
    flow = x['flow']
    
    if flow == 'In':
        return bud
    elif flow == 'Out':
        return bud * -1
    else:
        return 0
    

trans['budget'] = trans.apply(lambda x: correct_bud(x), axis = 1)

In [14]:
#create the function to get the difference between budget and actual
def make_diff(x):
    act = x['actual']
    bud = x['budget']
    flow = x['flow']
    
    if flow == 'In':
        diff = act - bud
    elif flow == 'Out':
        diff = bud - act
    else:
        diff = 0
    
    return diff

trans['diff'] = trans.apply(lambda x: make_diff(x), axis = 1)

In [15]:
trans

,month,category,group,flow,actual,budget,monthnum,diff
0,Apr,Alcohol,Leisure,Out,-86.08,-50.0,4,36.08
1,Apr,BB Income,Income,In,20.00,100.0,4,-80.00
2,Apr,Car,Leisure,Out,-203.94,-450.0,4,-246.06
3,Apr,Cellphone,Housing,Out,-168.68,-50.0,4,118.68
4,Apr,Clothes,Living,Out,-204.54,-200.0,4,4.54
...,...,...,...,...,...,...,...,...
212,Jul,Public Transportation,Leisure,Out,0.00,-20.0,7,-20.00
213,Jul,Special Splurge,Leisure,Out,0.00,-0.0,7,-0.00
214,Jul,Income Taxes,Living,Out,0.00,-0.0,7,-0.00
215,Jul,Viv's Choice,Leisure,Out,0.00,-70.0,7,-70.00


# Pivot account table

In [16]:
acc_bal

,Accounts,2019,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,TD - Shared cq,4796.85,5504.23,5018.38,4306.79,3641.07,4786.64,3366.54,NaN,NaN,NaN,NaN,NaN,NaN
1,Tang - Shared cq,10491.20,8314.76,6169.55,6740.49,6723.89,6451.65,3888.13,NaN,NaN,NaN,NaN,NaN,NaN
2,Tang - Gods cq,394.63,334.63,240.00,0.00,778.29,625.15,950.00,NaN,NaN,NaN,NaN,NaN,NaN
3,Tang - sv,41.33,64.70,50.00,62.65,86.82,111.11,50.00,NaN,NaN,NaN,NaN,NaN,NaN
4,Tang - TFSA Inv,5476.75,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
5,Tang - TFSA Inv - int,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
6,WS - Viv Inv,16244.81,19137.19,19537.19,21846.35,25337.19,25737.19,26137.16,NaN,NaN,NaN,NaN,NaN,NaN
7,WS - Viv Inv - int,0.00,1745.61,1606.68,2225.69,2265.51,2685.90,3369.14,NaN,NaN,NaN,NaN,NaN,NaN
8,WS - Timmy Inv,15238.99,13526.88,13503.72,13680.37,14103.72,14404.22,14703.72,NaN,NaN,NaN,NaN,NaN,NaN
9,WS - Timmy Inv - int,0.00,1819.60,2128.34,2736.73,2783.90,3059.21,3471.95,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
acc = acc_bal.melt('Accounts',['2019','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
acc.rename( columns= {"Accounts": 'account','variable':'month','value':'balance'},inplace = True)
acc.dropna(inplace = True)

In [18]:
acc

,account,month,balance
0,TD - Shared cq,2019,4796.85
1,Tang - Shared cq,2019,10491.20
2,Tang - Gods cq,2019,394.63
3,Tang - sv,2019,41.33
4,Tang - TFSA Inv,2019,5476.75
...,...,...,...
141,CIBC - Credit,Jun,0.00
142,EQ Chequing,Jun,3229.93
144,EQ TFSA GIC,Jun,6000.00
145,EQ TFSA SV,Jun,4078.45


### write all the tables to csv

In [19]:
writer = pd.ExcelWriter('Clean_Data.xlsx', engine='xlsxwriter')
acc.to_excel(writer,"Account", index = False)
trans.to_excel(writer,"Transactions",index = False)
writer.save()